# 5.2. Tratando as *Features* extraídas

* **Input**: Dados com as respostas de LLM não tratadas

* **Output**: Respostas de LLM tratas + variável com True/False da alternativa

## 5.2.1. Extração da tabela original

In [2]:
# Importando Dependências
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

In [3]:
# Leitura dos dados
enem_data = pd.read_pickle("../data/final/enem_data_embeddings.pkl")
enem_data.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,...,distratores_embbedings_word2vec_100,similaridade_enunciado_gabarito_100,similaridade_enunciado_distratores_100,similaridade_gabarito_distratores_100,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,...,"[0.041403063, -0.019931864, 0.13105607, -0.080...",0.236482,0.699529,0.379381,"[0.09772969, 0.013166125, -0.032748703, 0.0580...","[-0.04434625, 0.2105245, -0.18144351, 0.030336...","[0.12940213, 0.22066694, 0.0586984, 0.03139533...",0.251504,0.736054,0.426387
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,...,"[-0.008315361, 0.010459961, 0.008144858, 0.036...",0.517151,0.449300,0.560128,"[-0.018823402, 0.07832528, -0.016192827, 0.027...","[0.21008351, 0.5248949, 0.03625112, 0.01772637...","[0.078055575, 0.19092263, -0.0076030395, -0.03...",0.634785,0.504648,0.587013
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,...,"[0.2935421, -0.019240078, 0.0034812312, 0.0143...",0.546526,0.564335,0.681273,"[-0.013062097, 0.038470805, -0.020605344, 0.05...","[0.16413634, -0.035050992, 0.08454134, 0.23227...","[0.1313074, 0.058539692, 0.0961813, 0.17813754...",0.655402,0.634760,0.763059
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,...,"[0.05247575, 0.061523248, 0.010398333, 0.12163...",0.382210,0.558623,0.591672,"[-0.042833664, -0.01434522, -0.002431296, 0.01...","[0.102909006, 0.06116425, -0.13846825, 0.33034...","[0.11364358, 0.071867414, 0.046681672, 0.18046...",0.485125,0.628863,0.766145
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,...,"[0.13021359, 0.0043819174, -0.019453753, 0.061...",0.435788,0.628210,0.611719,"[0.010493249, 0.11095436, 0.017243173, 0.05319...","[0.267512, -0.081234336, 0.08019033, 0.1294613...","[0.13863416, 0.16260158, -0.0043905806, 0.2472...",0.559584,0.707497,0.692311


## 5.2.2. Extração das respostas do Llama

In [4]:
# leitura do csv

df_llama = pd.read_csv('../data/final/llama_extraction_csv.csv') 
df_llama.head()

,Unnamed: 0,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec,resposta_llamma,acerto_llamma
0,0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,[ 6.11259453e-02 -4.80512679e-02 -5.90382852e-...,[ 0.0875405 0.14673425 0.17936975 -0.033228...,C,True
1,1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,[ 9.6771289e-03 1.1954352e-01 2.1252066e-02 ...,[ 1.61291629e-01 2.37609118e-01 8.02588761e-...,D,True
2,2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,[ 0.06814931 -0.02395453 0.02739678 -0.003347...,[ 0.26195732 -0.13758467 -0.16108766 0.071330...,D.,False
3,3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,[ 0.01932281 -0.03491368 -0.00591799 0.012698...,[ 0.253879 -0.00519249 -0.02825475 0.109827...,E.,False
4,4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,[-3.24988738e-02 8.80865678e-02 -4.07014266e-...,[ 0.248065 -0.15562867 -0.114118 -0.121275...,C,True


In [5]:
# dropando coluna não utilizada

df_llama.drop('acerto_llamma', axis=1, inplace = True) 

In [6]:
# utilizando regex para padronizar respostas que a LLM deu

df_llama['resposta_llamma_limpo'] = df_llama['resposta_llamma'].str.findall(r'\b([A-E])\b').str[-1]

In [7]:
# gerando variável true/false de acertos do modelo Llama

df_llama["acerto_llamma"] = df_llama[["gabarito", "resposta_llamma_limpo"]].apply(
    lambda row: row["gabarito"] == row["resposta_llamma_limpo"], axis=1
)

In [8]:
# quantidade de acertos do Llama

df_llama["acerto_llamma"].sum()

154

In [9]:
# pegando apenas as colunas de interesse e as necessárias para o merge

df_llama = df_llama[['numero_questao','ANO','resposta_llamma_limpo','acerto_llamma']]

In [10]:
# padronizando nome

df_llama.rename(columns={'resposta_llamma_limpo': 'resposta_llama'}, inplace=True)


In [11]:
# verificando tabela final

df_llama.head()

,numero_questao,ANO,resposta_llama,acerto_llamma
0,1,2017,C,True
1,2,2017,D,True
2,3,2017,D,True
3,4,2017,E,True
4,5,2017,C,True


## 5.2.3. Extração das respostas do Deepseek

In [12]:
# leitura do csv

df_deepseek = pd.read_csv('../data/final/deepseek_extraction_csv.csv') 
df_deepseek.head()

,Unnamed: 0,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,enunciado_limpo,alternativas_limpo,gabarito_texto_limpo,distratores_limpo,enunciado_embeddings_word2vec,gabarito_embeddings_word2vec,resposta_deepseek,acerto_deepseek
0,0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,africano além astronomia centro cidade comérci...,A: isolamento ocidental saara; B: exploração i...,circulação posição redes relativa,competição econômica exploração intensiva isol...,[ 6.11259453e-02 -4.80512679e-02 -5.90382852e-...,[ 0.0875405 0.14673425 0.17936975 -0.033228...,A resposta correta é: C,False
1,1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,aberrações acontecimentos após assumida branco...,A: americana ataque base feito harbor japonese...,campos concentração eslavos execução guetos ju...,americana americanas ataque atômicas base bomb...,[ 9.6771289e-03 1.1954352e-01 2.1252066e-02 ...,[ 1.61291629e-01 2.37609118e-01 8.02588761e-...,D,True
2,2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,abstratas afetados agradar ação bentham condut...,A: fundamentação positivista viés; B: convençã...,caráter pragmático racionalidade,comportamental convenção fundamentação inclina...,[ 0.06814931 -0.02395453 0.02739678 -0.003347...,[ 0.26195732 -0.13758467 -0.16108766 0.071330...,D,True
3,3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,apresenta assembleia bases bem cidadão concepç...,A: educação escolar modernização; B: atualizaç...,civil igualdade princípio universalização,aristocráticos atualização conhecimento costum...,[ 0.01932281 -0.03491368 -0.00591799 0.012698...,[ 0.253879 -0.00519249 -0.02825475 0.109827...,E,True
4,4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,aplicação artigo bens brasil competindo consta...,A: etnia miscigenação racial; B: igualdade jur...,cultural espaço sobrevivência,ambiental bem econômica educação etnia igualda...,[-3.24988738e-02 8.80865678e-02 -4.07014266e-...,[ 0.248065 -0.15562867 -0.114118 -0.121275...,C,True


In [13]:
# dropando coluna não utilizada

df_deepseek.drop('acerto_deepseek', axis=1, inplace = True) 

In [14]:
# aqui visualizamos e tratamos manualmente algumas respostas mais complexas que o Deepseek forneceu

df_deepseek.loc[10, 'resposta_deepseek']

'Vamos analisar cada alternativa em relação ao problema apresentado no texto, que é a ineficiência logística da soja, causada principalmente por gargalos na infraestrutura de transporte (filas de navios, caminhões e trens):\n\n**A. Isenção de impostos de transporte**  \nNão resolve o problema estrutural de gargalos físicos (portos, ferrovias, etc.). A questão é a capacidade da infraestrutura, não o custo tributário.\n\n**B. Construção de terminais atracadouros**  \nSolução direta para o problema citado. O texto menciona filas de navios por falta de infraestrutura portuária. Terminais ampliariam a capacidade de atracagem, reduzindo esperas e custos logísticos.  \n\n**C. Diversificação de parceiros comerciais**  \nIrrelevante para o problema logístico interno. A questão é a infraestrutura física, não relações comerciais.  \n\n**D. Contratação de trabalhadores portuários**  \nNão resolve gargalos estruturais (como falta de terminais). O problema não é falta de mão de obra, mas sim de infr

In [15]:
df_deepseek.loc[10, 'resposta_deepseek'] = 'B'

In [16]:
df_deepseek.loc[41, 'resposta_deepseek']

'Vamos analisar as informações fornecidas e as alternativas:\n\n1. O enunciado descreve as temperaturas mínimas em diferentes regiões do Brasil e os índices de umidade relativa do ar.\n2. A pergunta específica é sobre o fator climático determinante para explicar os altos índices de umidade nas regiões Nordeste e Sul.\n\nAnalisando as alternativas:\n- A) Altitude: Não é o fator principal, pois ambas as regiões têm áreas de baixa altitude.\n- B) Vegetação: Embora possa influenciar, não é o principal fator para a umidade nessas regiões.\n- C) Massas de ar: Esta é a resposta correta, pois as massas de ar úmidas (como a massa Tropical Atlântica no Nordeste e a Polar Atlântica no Sul) são responsáveis pela alta umidade e precipitação nessas regiões.\n- D) Correntes marítimas: Influenciam, mas não são o fator principal.\n- E) Continentalidade: Não se aplica, pois ambas as regiões têm forte influência oceânica.\n\nA resposta correta é: C'

In [17]:
df_deepseek.loc[41, 'resposta_deepseek'] = 'C'

In [18]:
df_deepseek.loc[98, 'resposta_deepseek']

'Vamos analisar cada alternativa:\n\nA) Ciclo hidrológico - Não está relacionado à formação de continentes.\nB) Processo erosivo - Também não explica a formação de um supercontinente.\nC) Estrutura geológica - Esta é a correta, pois a formação e movimentação de continentes está diretamente relacionada aos processos geológicos e tectônicos.\nD) Índice pluviométrico - Não tem relação com a formação de continentes.\nE) Pressão atmosférica - Também não está relacionada.\n\nA resposta correta é: C'

In [19]:
df_deepseek.loc[98, 'resposta_deepseek'] = 'C'

In [20]:
df_deepseek.loc[103, 'resposta_deepseek']

'Vamos analisar o enunciado e as alternativas:\n\nO texto descreve como Newton, ao observar a queda da maçã, passou de uma explicação superficial (animais derrubando) para uma busca por princípios fundamentais (força subjetiva causando a queda). Isso representa o núcleo do método científico moderno - ir além das aparências para descobrir leis universais.\n\nAnalisando as alternativas:\nA) Falsificação - não é o foco (Newton não está testando/excluindo hipóteses)\nB) Negação da observação - incorreto (ele parte da observação)\nC) Proposição de hipóteses - parcialmente correto, mas não capta a essência\nD) Contemplação - muito passivo (Newton está ativamente buscando princípios)\nE) Universalização - CORRETO (ele busca uma lei geral a partir do caso específico)\n\nA resposta correta é: E'

In [21]:
df_deepseek.loc[103, 'resposta_deepseek'] = 'E'

In [22]:
df_deepseek.loc[127, 'resposta_deepseek']

'Vamos analisar cada alternativa:\n\nA) "promoção de atos ecumênicos" - O texto não menciona reconciliação entre diferentes religiões.\nB) "fomento de orientações bíblicas" - Embora haja adaptação, não é o foco principal.\nC) "apropriação de cerimônias seculares" - Corresponde à absorção de rituais pagãos.\nD) "retomada de ensinamentos apostólicos" - Não há referência a ensinos originais dos apóstolos.\nE) "ressignificação de rituais fundamentalistas" - O termo "fundamentalistas" não se aplica ao contexto.\n\nA alternativa que melhor descreve o processo de assimilação de rituais pagãos (como as fogueiras) pela Igreja é:\n\nC'

In [23]:
df_deepseek.loc[127, 'resposta_deepseek'] = 'C'

In [24]:
df_deepseek.loc[139, 'resposta_deepseek']

'Vamos analisar o texto e as alternativas:\n\nO texto descreve um modelo de planejamento urbano que:\n- Incentiva o crescimento urbano a qualquer custo\n- Vê as cidades como máquinas de produzir riqueza\n- Foca em "apertar a máquina" (otimização econômica)\n\nIsso caracteriza um modelo:\n- Não participativo (exclui gestão popular - A)\n- Não sustentável (exclui B)\n- Não focado em bem-estar social (exclui C)\n- Centralizado no poder econômico, não governamental (exclui D)\n\nA alternativa que melhor se encaixa é:\nE - ampliação da participação empresarial (o texto enfatiza a lógica empresarial de produção de riqueza)\n\nResposta: E'

In [25]:
df_deepseek.loc[139, 'resposta_deepseek'] = 'E'

In [26]:
df_deepseek.loc[158, 'resposta_deepseek']

'Vamos analisar o texto e as alternativas:\n\nO texto descreve a poluição do rio Tietê, especificamente mencionando:\n1. A água não tem oxigênio suficiente para sustentar vida\n2. A poluição ocorre ao longo de uma área urbana/metropolitana\n3. A causa está relacionada com a falta de oxigênio na água\n\nAnalisando as alternativas:\nA) Não menciona sítio urbano - irrelevante\nB) Extinção da vegetação não é a causa direta da falta de oxigênio\nC) Carga de dejetos (esgoto) é a principal causa da falta de oxigênio em rios urbanos\nD) Captação de água pluvial não causa falta de oxigênio\nE) Defensivos agrícolas não são mencionados como causa\n\nA causa mais direta para a falta de oxigênio em rios urbanos é o lançamento de esgoto (carga orgânica que consome oxigênio na decomposição).\n\nResposta correta: C'

In [27]:
df_deepseek.loc[158, 'resposta_deepseek'] = 'C'

In [28]:


df_deepseek.loc[176, 'resposta_deepseek']

'Vamos analisar cada alternativa:\n\nA) A estagnação de direitos adquiridos e o anacronismo da legislação vigente são mencionados, mas não explicam a desigualdade de oportunidades.\n\nB) A manutenção do status quo gerencial e dos padrões de socialização familiar são fatores que perpetuam a desigualdade de oportunidades, pois mantêm as estruturas que limitam as mulheres.\n\nC) A desestruturação da herança patriarcal e mudanças no perfil ocupacional seriam positivas, não explicando a desigualdade.\n\nD) A disputa na composição sindical e presença política são relevantes, mas não são a causa central da desigualdade de oportunidades.\n\nE) As exigências de aperfeiçoamento profissional e habilidades gerenciais são consequências, não causas da desigualdade.\n\nA alternativa que melhor explica a persistência da desigualdade de oportunidades é a B, pois aponta para a manutenção das estruturas sociais e gerenciais que perpetuam a desigualdade.\n\nB'

In [29]:
df_deepseek.loc[176, 'resposta_deepseek'] = 'B'

In [30]:
df_deepseek.loc[186, 'resposta_deepseek']

'Pela análise do texto, a situação descrita refere-se a pessoas que não têm nacionalidade reconhecida por nenhum país, o que caracteriza a condição de apátrida. Portanto, a alternativa correta é:\n\nB'

In [31]:
df_deepseek.loc[186, 'resposta_deepseek'] = 'B'

In [32]:
df_deepseek.loc[203, 'resposta_deepseek']

'Analisando o enunciado, o fator determinante para a instalação das refinarias de petróleo é a proximidade a espaços com infraestrutura de circulação, pois isso facilita o transporte tanto da matéria-prima quanto dos produtos finais.\n\nA resposta correta é: E'

In [33]:
df_deepseek.loc[203, 'resposta_deepseek'] = 'E'

In [34]:
df_deepseek.loc[239, 'resposta_deepseek']

'O texto discute a natureza do perdão, destacando que ele não é garantido e depende da autonomia de quem o concede. A alternativa que melhor reflete essa ideia é a que enfatiza a autonomia dos indivíduos nas relações interpessoais. Portanto, a resposta correta é:\n\nE'

In [35]:
df_deepseek.loc[239, 'resposta_deepseek'] = 'E'

In [36]:
df_deepseek.loc[244, 'resposta_deepseek']

'Analisando o texto e o contexto histórico da Declaração Balfour, o objetivo declarado era estabelecer um lar nacional para o povo judeu na Palestina, sem prejudicar os direitos das comunidades não judias. No entanto, o processo resultou em conflitos territoriais entre judeus e árabes, demonstrando a incapacidade do governo britânico de mediar esses conflitos de forma efetiva.\n\nA alternativa correta é: C'

In [37]:
df_deepseek.loc[244, 'resposta_deepseek'] = 'C'

In [38]:
# agora, tratamos de forma mais geral as respostas menos complexas

df_deepseek['resposta_deepseek_limpo'] = df_deepseek['resposta_deepseek'].str.findall(r'\b([A-E])\b').str[-1]

In [39]:
# gerando variável de true/false de acertos do modelo Deepseek

df_deepseek["acerto_deepseek"] = df_deepseek[["gabarito", "resposta_deepseek_limpo"]].apply(
    lambda row: row["gabarito"] == row["resposta_deepseek_limpo"], axis=1
)

In [40]:
# quantidade de acertos do Deepseek

df_deepseek["acerto_deepseek"].sum()

223

In [41]:
# pegando apenas as colunas de interesse e as necessárias para o merge

df_deepseek = df_deepseek[['numero_questao','ANO','resposta_deepseek_limpo','acerto_deepseek']]

In [42]:
# padronizando nome

df_deepseek.rename(columns={'resposta_deepseek_limpo': 'resposta_deepseek'}, inplace=True)


In [43]:
# verificando tabela final

df_deepseek.head()

,numero_questao,ANO,resposta_deepseek,acerto_deepseek
0,1,2017,C,True
1,2,2017,D,True
2,3,2017,D,True
3,4,2017,E,True
4,5,2017,C,True


## 5.2.4. Extração das respostas do Deepseek

In [44]:
# inserindo as novas variáveis do Llama no dataset original

df_merged = pd.merge(
    enem_data,                 
    df_llama,                 
    on=['numero_questao', 'ANO'],  
    how='left'           
)

In [45]:
# inserindo as novas variáveis do Deepseek no dataset original

df_merged = pd.merge(
    df_merged,                 
    df_deepseek,                 
    on=['numero_questao', 'ANO'],  
    how='left'           
)

In [46]:
# resultado final

df_merged.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,...,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50,resposta_llama,acerto_llamma,resposta_deepseek,acerto_deepseek
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,...,"[0.09772969, 0.013166125, -0.032748703, 0.0580...","[-0.04434625, 0.2105245, -0.18144351, 0.030336...","[0.12940213, 0.22066694, 0.0586984, 0.03139533...",0.251504,0.736054,0.426387,C,True,C,True
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,...,"[-0.018823402, 0.07832528, -0.016192827, 0.027...","[0.21008351, 0.5248949, 0.03625112, 0.01772637...","[0.078055575, 0.19092263, -0.0076030395, -0.03...",0.634785,0.504648,0.587013,D,True,D,True
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,...,"[-0.013062097, 0.038470805, -0.020605344, 0.05...","[0.16413634, -0.035050992, 0.08454134, 0.23227...","[0.1313074, 0.058539692, 0.0961813, 0.17813754...",0.655402,0.634760,0.763059,D,True,D,True
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,...,"[-0.042833664, -0.01434522, -0.002431296, 0.01...","[0.102909006, 0.06116425, -0.13846825, 0.33034...","[0.11364358, 0.071867414, 0.046681672, 0.18046...",0.485125,0.628863,0.766145,E,True,E,True
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,...,"[0.010493249, 0.11095436, 0.017243173, 0.05319...","[0.267512, -0.081234336, 0.08019033, 0.1294613...","[0.13863416, 0.16260158, -0.0043905806, 0.2472...",0.559584,0.707497,0.692311,C,True,C,True


In [47]:
# salvando pickle
df_merged.to_pickle("../data/final/enem_data_final.pkl")